In [14]:
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def solution_model():

    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
          if(logs.get('accuracy') > 0.9):
              print("Reached 99% accuracy so cancelling trining!")
              self.model.stop_training = True

    callbacks = myCallback()
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
    urllib.request.urlretrieve(url, 'sarcasm.json')

    # DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
    vocab_size = 1000
    embedding_dim = 16
    max_length = 120
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"
    training_size = 20000

    sentences = []
    labels = []
    # YOUR CODE HERE

    with open('sarcasm.json', 'r') as f:
      data = json.load(f)

    for d in data:
      sentences.append(d['headline'])
      labels.append(d['is_sarcastic'])

    train_ratio = 0.8
    train_size = int(len(data) * train_ratio)

    # train 분할
    train_sentences = sentences[:train_size]
    valid_sentences = sentences[train_size:]
    # label 분할
    train_labels = labels[:train_size]
    valid_labels = labels[train_size:]

    token = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
    token.fit_on_texts(sentences)
    word_index = token.word_index

    train_sequences = token.texts_to_sequences(train_sentences)
    valid_sequences = token.texts_to_sequences(valid_sentences)

    train_padded = pad_sequences(train_sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)
    valid_padded = pad_sequences(valid_sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length)

    train_labels = np.asarray(train_labels)
    valid_labels = np.asarray(valid_labels)

    model = tf.keras.Sequential([
    # YOUR CODE HERE. KEEP THIS OUTPUT LAYER INTACT OR TESTS MAY FAIL
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        # tf.keras.layers.Flatten(),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(train_padded, train_labels, 
                    validation_data=(valid_padded, valid_labels),
                    epochs=5, 
                    verbose=2,
                    callbacks=[callbacks])
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")


Epoch 1/5
668/668 - 2s - loss: 0.6657 - accuracy: 0.5840 - val_loss: 0.6014 - val_accuracy: 0.7606
Epoch 2/5
668/668 - 1s - loss: 0.4934 - accuracy: 0.7821 - val_loss: 0.4451 - val_accuracy: 0.7988
Epoch 3/5
668/668 - 1s - loss: 0.4089 - accuracy: 0.8183 - val_loss: 0.4192 - val_accuracy: 0.8055
Epoch 4/5
668/668 - 1s - loss: 0.3873 - accuracy: 0.8234 - val_loss: 0.4119 - val_accuracy: 0.8091
Epoch 5/5
668/668 - 1s - loss: 0.3736 - accuracy: 0.8307 - val_loss: 0.3981 - val_accuracy: 0.8173


In [15]:
from google.colab import files
files.download("mymodel.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>